In [2]:
import numpy as np
from typing import Union
import math
import time

In [3]:
"""
Given an integer B as represented by two arrays a, b
where B = the product over i of (a_i ^ b_i)
return the number of positive integers 
x less than B such that gcd(x, B) = 1
"""
def phi(a: np.array, b: np.array) -> int:
    
    result = 1
    
    for prime,power in zip(a,b):
        
        if power != 0:
        
            result *= (prime**(power-1)) * (prime - 1)
            
    return result

In [4]:
phi([2,3,5,7,11,13], [1,0,0,1,0,0])

6

In [5]:
"""
Given a factor base `factor_base` and an integer a, check whether or not 
a is B-smooth

returns a list `exponents` where each element 
exponents[i] is the exponent of the prime factor_base[i] 
if a is B-smooth

returns an empty array otherwise
"""

def is_B_smooth(a: int, factor_base: np.array) -> Union[np.array, bool]:
    
    exponents = np.zeros_like(factor_base)
    
    for i,p in enumerate(factor_base):
        
        if a == 1:          
            break
        
        while a % p == 0:         
            a = a / p           
            exponents[i] += 1
        
    if a > 1:
        return np.array([])
    
    return exponents

In [6]:
'''
Given a vector of exponents, return the vector mod-2 
'''
def exponent_vector_mod_2(exponents: np.array) -> np.array:
    
    return exponents % 2

In [7]:
"""
Given number to factor N pick a B value for factor base
Taken from paper linked in the chat
"""

def choose_B(N: int) -> int:
    L = np.e ** (0.5 * (np.log(N) * np.log(np.log(N)))**0.5)
    return int(np.ceil(L))
    

In [8]:
"""
Given an integer B, use the sieve of Eratosthenes to
find its prime factors 

returns a list of prime factors less than B

"""

def create_factors(n):
    flags = np.ones(n, dtype=bool)
    flags[0] = flags[1] = False
    for i in range(2, int(np.sqrt(n)) + 1):
        if flags[i]:
            flags[i*i::i] = False
    return np.flatnonzero(flags)

In [9]:
B = choose_B(16921456439215439701)

B

651

In [10]:
create_factors(B)

array([  2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41,
        43,  47,  53,  59,  61,  67,  71,  73,  79,  83,  89,  97, 101,
       103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
       173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239,
       241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,
       317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397,
       401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467,
       479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569,
       571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643,
       647])

In [11]:
from numpy.linalg import svd

def nullspace(A, atol=1e-13, rtol=0):
    A = np.atleast_2d(A)
    u, s, vh = svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

In [12]:
def gaussian_elimination_mod2(M):

    contains_pivot = [False]*len(M)
    pivot_found = False
    pivot_loc = {}
    
    for j in range(len(M[0])):
        pivot_found = False
        #Look for pivot
        for i in range(len(M)):
            # Pivot Found at row i and column j
            if(M[i][j] == 1):
                contains_pivot[i] = True
                pivot_loc[j]=i
                pivot_found = True
                break
          
        if pivot_found:
            for k in range(len(M[0])):
                if (k == j):
                    continue
                if (M[i][k] == 1):
                    for l in range(len(M)):
                        M[l][k] = (M[l][j] + M[l][k])%2
                        
    return M, contains_pivot, pivot_loc

# P3(b) find the left null space of M mod 2
def left_null_space_mod2(M): 
    # solve with gaussian elimination
    M, contains_pivot, pivot_loc = gaussian_elimination_mod2(M)
    result = []
    # Find left null space
    for i in range(len(M)):
        #Find dependent rows
        null_space = []
        if not contains_pivot[i]:
            row = M[i]
            null_space = [i]
            for j in range(len(row)):
                if (M[i][j]==1):
                    null_space.append(pivot_loc[j])
            result.append(null_space)
    # Encoding the indicies
    out_vecs = []
    for space in result:
        vec = [0]*len(M)
        for idx in space:
            vec[idx] = 1
        out_vecs.append(vec)
    return out_vecs

In [23]:
num_multiples_of_root_n = 10
'''
Given an integer n, try to factor it into two large primes
Returns a tuple of the factors if able to factor
Returns false if couldn't factor
''' 
def quadratic_sieve(n: int) -> Union[tuple[int, int], bool]:
    
    
    """
    Choose smoothness bound B
    """
    
    B = choose_B(n)
    factor_base = create_factors(B)
    
    
    # represent B as its prime factorization

    B_primes = factor_base
    B_exp = is_B_smooth(B,factor_base)
    
    if (len(B_exp) == 0):
        # B is a prime itself
        B_primes = [B]
        B_exp = [1]
    

    """
    Find phi(B) + 1 numbers a_i such that 
        b_i = a_i^2 (mod n)
    is B-smooth.
    
    In checking if b_i is B-smooth, we also
    factor b_i.
    
    """
    # assumes B is prime
    phi_B = phi(B_primes, B_exp)
    print(B,phi_B)
    
    all_b_exponents = []
    all_a = []
    all_b = []
    
    
    
    all_times_between_numbers_found = []
    iteration = 1

    last_added_start = time.time()
    
    root_n = np.sqrt(n)
    
    while len(all_b_exponents) < phi_B + 1:
        
        
        for i in range(1,10):
            # find a new entry to add to the list
            
            curr_a = math.ceil(np.sqrt(i)*root_n) + iteration
            
            
            curr_b = curr_a**2 % n
            
            #print(i,curr_b,iteration)

            start = time.time()
            curr_exponents = is_B_smooth(curr_b, factor_base)
            
            
            end = time.time()

            #print("time to compute is_B_smooth:", end-start)
            if (len(curr_exponents) > 0):
                print(len(all_b_exponents))


                all_a.append(curr_a)
                all_b.append(curr_b)
                all_b_exponents.append(curr_exponents) 

                # print(len(all_b_exponents))
                last_added_end = time.time()

                print("time between subsequent numbers found:", last_added_end-last_added_start)

#                 all_times_between_numbers_found.append(last_added_end - last_added_start)
                last_added_start = time.time()
        
        iteration += 1
        
                
    """
    Generate exponent vectors 
    (mod 2) for each b_i
    """
    
    
    all_exponent_vectors = [ exponent_vector_mod_2(b_exponents) for b_exponents in all_b_exponents ]
    
    """
    Find linear combination of these vectors that
    sums to the zero vector
    """
    
    M = np.array(all_exponent_vectors) # matrix of exponent vectors
    
    zero_vector = np.zeros(M.shape[1])
    
#     try:
#         x = np.linalg.solve(M, zero_vector)
#     except np.linalg.LinAlgError:
#         x = np.zeros(M.shape[0])
#         x[0] = 1
    
    
    
    # solutions = nullspace(M.T)
    # solutions *= np.min(solutions)
    solutions = left_null_space_mod2(M)
    """
    Multiply the corresponding a_i for the above 
    combination and call the result (mod n) 'a'
    
    Multiply the corresponding b_i for the above 
    combination and call the result (mod n) 'b'
    
    """
    tol = 1e-13
    for j in solutions:
        print("\n",j)
        x = 1
        y = 1
        for i, k in enumerate(j):
            
            if (k > tol):
                x *= all_a[i]
                y *= all_b[i]
                
                print(all_a[i],all_b[i])
        x = x % n
        y = int(np.sqrt(y)) % n
        factor = np.gcd(x-y, n)
        if (factor != 1 and factor != n):
            return factor, int(n / factor)

    return 
    
    
    
    

In [14]:
quadratic_sieve(539873)



19 18
0
1823
3935
3457
4349
0
3296
6018
6006
7292
1
4771
8103
8557
10237
1
6248
10190
11110
13184
1
7727
12279
13665
16133
1
9208
14370
16222
19084
1
10691
16463
18781
22037
1
12176
18558
21342
24992
1
13663
20655
23905
27949
2
15152
22754
26470
30908
2
16643
24855
29037
33869
2
18136
26958
31606
36832
2
19631
29063
34177
39797
2
21128
31170
36750
42764
3
22627
33279
39325
45733
5
24128
35390
41902
48704
5
25631
37503
44481
51677
5
27136
39618
47062
54652
5
28643
41735
49645
57629
5
30152
43854
52230
60608
5
31663
45975
54817
63589
5
33176
48098
57406
66572
5
34691
50223
59997
69557
5
36208
52350
62590
72544
5
37727
54479
65185
75533
5
39248
56610
67782
78524
5
40771
58743
70381
81517
5
42296
60878
72982
84512
5
43823
63015
75585
87509
5
45352
65154
78190
90508
6
46883
67295
80797
93509
6
48416
69438
83406
96512
6
49951
71583
86017
99517
6
51488
73730
88630
102524
6
53027
75879
91245
105533
6
54568
78030
93862
108544
7
56111
80183
96481
111557
7
57656
82338
99102
114572
7
59203
84495
1

(277, 1949)

In [ ]:
quadratic_sieve(16921456439215439701)

651 360
0
time between subsequent numbers found: 0.02377009391784668
1
time between subsequent numbers found: 0.004499912261962891
2
time between subsequent numbers found: 0.013064384460449219
3
time between subsequent numbers found: 0.04648399353027344
4
time between subsequent numbers found: 0.04665803909301758
5
time between subsequent numbers found: 0.08608698844909668
6
time between subsequent numbers found: 0.019078969955444336
7
time between subsequent numbers found: 0.06000518798828125
8
time between subsequent numbers found: 0.03736424446105957
9
time between subsequent numbers found: 0.016240835189819336
10
time between subsequent numbers found: 0.16729211807250977
11
time between subsequent numbers found: 0.06911993026733398
12
time between subsequent numbers found: 0.002804994583129883
13
time between subsequent numbers found: 0.003002166748046875
14
time between subsequent numbers found: 0.14419102668762207
15
time between subsequent numbers found: 0.009636878967285156
16


134
time between subsequent numbers found: 0.22467923164367676
135
time between subsequent numbers found: 0.40116310119628906
136
time between subsequent numbers found: 0.4149661064147949
137
time between subsequent numbers found: 0.8387751579284668
138
time between subsequent numbers found: 0.6902070045471191
139
time between subsequent numbers found: 0.0053730010986328125
140
time between subsequent numbers found: 0.8622877597808838
141
time between subsequent numbers found: 0.2344210147857666
142
time between subsequent numbers found: 0.8859829902648926
143
time between subsequent numbers found: 0.39144301414489746
144
time between subsequent numbers found: 0.344480037689209
145
time between subsequent numbers found: 0.2474040985107422
146
time between subsequent numbers found: 0.11268806457519531
147
time between subsequent numbers found: 0.24709081649780273
148
time between subsequent numbers found: 0.08061790466308594
149
time between subsequent numbers found: 2.7863218784332275


270
time between subsequent numbers found: 3.488391876220703
271
time between subsequent numbers found: 1.3552677631378174
272
time between subsequent numbers found: 3.5514280796051025
273
time between subsequent numbers found: 6.653807163238525
274
time between subsequent numbers found: 0.721754789352417
275
time between subsequent numbers found: 1.1887669563293457
276
time between subsequent numbers found: 0.35666394233703613
277
time between subsequent numbers found: 1.5284039974212646
278
time between subsequent numbers found: 0.502971887588501
279
time between subsequent numbers found: 0.28217482566833496
280
time between subsequent numbers found: 0.844789981842041
281
time between subsequent numbers found: 2.3258156776428223
282
time between subsequent numbers found: 0.049566030502319336
283
time between subsequent numbers found: 1.033864974975586
284
time between subsequent numbers found: 0.26746702194213867
285
time between subsequent numbers found: 1.3631789684295654
286
time 

In [341]:
print(-5 % 3)

1


In [14]:
np.sqrt(16921456439215439701)


4113569792.6758747

In [15]:
4113569793 + 1

4113569794

In [18]:
4113569793**2

16921456441882062849

In [19]:
16921456441882062849 % 16921456439215439701

2666623148